# POSTFILE Analysis with PyAERMOD

This notebook demonstrates how to work with AERMOD POSTFILE output:

1. Parse text (PLOT) format POSTFILEs
2. Parse binary (UNFORM) format POSTFILEs
3. Handle deposition data in binary POSTFILEs
4. Extract time-series at specific receptors
5. Export results to pandas DataFrames

**Requirements:**
- PyAERMOD installed (`pip install pyaermod`)

In [ ]:
import struct
import tempfile
from pathlib import Path

from pyaermod.postfile import read_postfile

print("PyAERMOD postfile module loaded successfully!")

## 1. Text (PLOT) Format POSTFILEs

Text POSTFILEs are the default format and contain header comments followed by data rows.

In [ ]:
# Create a synthetic text POSTFILE for demonstration
text_content = """* AERMOD ( 24142):  Postfile Analysis Demo
* MODELING OPTIONS USED:  CONC   FLAT
* AVERTIME: 1-HR
* POLLUTID: SO2
* SRCGROUP: ALL
*    X         Y        CONC    ZELEV    ZHILL   ZFLAG   AVE   GRP     DATE(CONC)
*  (met)     (met)   (ug/m^3)   (m)      (m)             (hr)        (YYMMDDHH)
    100.000   200.000  12.34500   0.00     0.00    0.00     1   ALL    01010101
    300.000   400.000   8.76500   0.00     0.00    0.00     1   ALL    01010101
    500.000   600.000  15.23400   0.00     0.00    0.00     1   ALL    01010101
    100.000   200.000  10.50000   0.00     0.00    0.00     1   ALL    01010102
    300.000   400.000   6.30000   0.00     0.00    0.00     1   ALL    01010102
    500.000   600.000  13.10000   0.00     0.00    0.00     1   ALL    01010102
"""

# Write to temp file
tmpdir = Path(tempfile.mkdtemp())
text_file = tmpdir / "demo_text.pst"
text_file.write_text(text_content)

# Parse it
result = read_postfile(str(text_file))

print("Header Information:")
print(f"  Pollutant: {result.header.pollutant_id}")
print(f"  Averaging period: {result.header.averaging_period}")
print(f"  Source group: {result.header.source_group}")
print(f"  Records: {len(result.data)}")
print(f"  Max concentration: {result.max_concentration:.4f}")
print(f"  Max location: {result.max_location}")

In [ ]:
# Convert to DataFrame for analysis
df = result.to_dataframe()
print("DataFrame:")
print(df)

## 2. Binary (UNFORM) Format POSTFILEs

Binary POSTFILEs are more compact and use Fortran unformatted sequential records.
PyAERMOD auto-detects the format.

In [ ]:
def build_binary_record(kurdat, ianhrs, grpid, concentrations):
    """Build a Fortran unformatted sequential record."""
    grpid_bytes = grpid.ljust(8)[:8].encode("ascii")
    payload = struct.pack("<i", kurdat)
    payload += struct.pack("<i", ianhrs)
    payload += grpid_bytes
    for c in concentrations:
        payload += struct.pack("<d", c)
    rec_len = len(payload)
    return struct.pack("<i", rec_len) + payload + struct.pack("<i", rec_len)

# Create synthetic binary postfile with 4 receptors, 3 timesteps
records = b""
records += build_binary_record(2001010101, 1, "ALL", [1.5, 3.2, 5.7, 2.1])
records += build_binary_record(2001010102, 1, "ALL", [2.0, 4.1, 6.3, 3.0])
records += build_binary_record(2001010103, 1, "ALL", [1.8, 3.5, 5.0, 2.5])

binary_file = tmpdir / "demo_binary.pst"
binary_file.write_bytes(records)

# Parse — auto-detects binary format
result = read_postfile(str(binary_file))

print(f"Format detected: binary (UNFORM)")
print(f"Records: {len(result.data)}")
print(f"Unique timesteps: {result.data['date'].nunique()}")
print(f"Max concentration: {result.max_concentration:.4f}")
print()
print(result.to_dataframe())

## 3. Binary Deposition POSTFILEs

Deposition runs produce binary POSTFILEs with 3N floats per record:
- `CONC[1..N]` — concentration values
- `DRY_DEP[1..N]` — dry deposition values  
- `WET_DEP[1..N]` — wet deposition values

Use `has_deposition=True` to parse these.

In [ ]:
def build_deposition_record(kurdat, ianhrs, grpid, concs, dry_deps, wet_deps):
    """Build a binary record with deposition data (3xN floats)."""
    grpid_bytes = grpid.ljust(8)[:8].encode("ascii")
    payload = struct.pack("<i", kurdat)
    payload += struct.pack("<i", ianhrs)
    payload += grpid_bytes
    for c in concs:
        payload += struct.pack("<d", c)
    for d in dry_deps:
        payload += struct.pack("<d", d)
    for w in wet_deps:
        payload += struct.pack("<d", w)
    rec_len = len(payload)
    return struct.pack("<i", rec_len) + payload + struct.pack("<i", rec_len)

# 3 receptors with concentration + deposition
dep_records = b""
dep_records += build_deposition_record(
    2001010101, 1, "ALL",
    concs=[10.0, 20.0, 30.0],
    dry_deps=[0.10, 0.20, 0.30],
    wet_deps=[0.01, 0.02, 0.03],
)

dep_file = tmpdir / "demo_depo.pst"
dep_file.write_bytes(dep_records)

# Parse with deposition flag
dep_result = read_postfile(str(dep_file), has_deposition=True)
dep_df = dep_result.to_dataframe()

print("Deposition POSTFILE:")
print(dep_df[["x", "y", "concentration", "dry_depo", "wet_depo"]])

## 4. Timestep Queries

Extract data for specific timesteps or receptor locations.

In [ ]:
# Re-parse the binary file with multiple timesteps
result = read_postfile(str(binary_file))

# Get data for a specific timestep
ts1 = result.get_timestep("2001010101")
print("Timestep 2001010101:")
print(ts1)
print()

# Get all unique dates
dates = result.data["date"].unique()
print(f"Available timesteps: {list(dates)}")

## Summary

Key functions:
- `read_postfile(path)` — auto-detects text vs binary format
- `read_postfile(path, has_deposition=True)` — parse deposition data
- `result.to_dataframe()` — convert to pandas DataFrame
- `result.get_timestep(date)` — filter to specific time
- `result.max_concentration` / `result.max_location` — quick stats